# Train and Fine-Tune Sentence Transformers Models - Notebook Companion

In [ ]:
%%capture
!pip install sentence-transformers

In [5]:
import numpy as np
import pandas as pd
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


dataset = pd.read_csv("/content/atco2_train_data.csv")
column_name = "Transcription"

def generate_pairs(anchor_text):

    prompt = f"""Generate positive and negative sentence pairs for training a sentence transformer on ATC communications.

Anchor: {anchor_text}

Please provide:
1. One positive sentence (semantically similar, same meaning expressed differently)
2. One negative sentence (different meaning, but still ATC-related)

Format as JSON:
{{"positive": "...", "negative": "..."}}"""

    messages = [{"role": "user", "content": prompt}]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True
    )

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
    content = tokenizer.decode(output_ids, skip_special_tokens=True)

    return content


# Load existing pairs if file exists
import os
output_file = 'sentence_transformer_pairs.json'
training_pairs = []

if os.path.exists(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        training_pairs = json.load(f)
    print(f"Loaded {len(training_pairs)} existing pairs")

# Get already processed indices
processed_indices = {pair['anchor_id'] for pair in training_pairs}

for index, anchor_text in dataset[column_name].items():
    # Skip if already processed
    if index in processed_indices:
        print(f"Skipping row {index} (already processed)")
        continue

    print(f"Processing row {index}: {anchor_text[:50]}...")

    try:
        pairs_content = generate_pairs(anchor_text)

        try:
            import re
            json_match = re.search(r'\{.*\}', pairs_content, re.DOTALL)
            if json_match:
                pairs_data = json.loads(json_match.group())

                new_pair = {
                    "anchor": anchor_text,
                    "positive": pairs_data.get("positive", ""),
                    "negative": pairs_data.get("negative", ""),
                    "anchor_id": index
                }
            else:
                new_pair = {
                    "anchor": anchor_text,
                    "positive": pairs_content[:100] + "...",
                    "negative": "Failed to parse",
                    "anchor_id": index
                }
        except json.JSONDecodeError:
            new_pair = {
                "anchor": anchor_text,
                "positive": pairs_content,
                "negative": "JSON parse error",
                "anchor_id": index
            }

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        new_pair = {
            "anchor": anchor_text,
            "positive": "Generation failed",
            "negative": "Generation failed",
            "anchor_id": index
        }


    training_pairs.append(new_pair)


    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(training_pairs, f, indent=2, ensure_ascii=False)

    print(f"Saved progress: {len(training_pairs)} pairs completed")

print(f"Generated {len(training_pairs)} anchor-positive-negative triplets")
print("Sample triplet:")
if training_pairs:
    sample = training_pairs[-1]  # Show the last processed one
    print(f"Anchor: {sample['anchor']}")
    print(f"Positive: {sample['positive']}")
    print(f"Negative: {sample['negative']}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyError: 'text'

In [3]:
# import numpy as np
# import pandas as pd
# import json
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch


# model_name = "Qwen/Qwen2.5-7B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )


# dataset = pd.read_csv("/content/atco2_train_data.csv")
# column_name = "Transcription"

# def generate_pairs(anchor_text):

#     prompt = f"""Generate positive and negative sentence pairs for training a sentence transformer on ATC communications.

# Anchor: {anchor_text}

# Please provide:
# 1. One positive sentence (semantically similar, same meaning expressed differently)
# 2. One negative sentence (different meaning, but still ATC-related)

# Format as JSON:
# {{"positive": "...", "negative": "..."}}"""

#     messages = [{"role": "user", "content": prompt}]

#     text = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True,
#     )

#     model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

#     generated_ids = model.generate(
#         **model_inputs,
#         max_new_tokens=256,
#         temperature=0.7,
#         do_sample=True
#     )

#     output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
#     content = tokenizer.decode(output_ids, skip_special_tokens=True)

#     return content


# training_pairs = []
# for index, anchor_text in dataset[column_name].items():
#     print(f"Processing row {index}: {anchor_text[:50]}...")

#     try:
#         pairs_content = generate_pairs(anchor_text)


#         try:

#             import re
#             json_match = re.search(r'\{.*\}', pairs_content, re.DOTALL)
#             if json_match:
#                 pairs_data = json.loads(json_match.group())

#                 training_pairs.append({
#                     "anchor": anchor_text,
#                     "positive": pairs_data.get("positive", ""),
#                     "negative": pairs_data.get("negative", ""),
#                     "anchor_id": index
#                 })
#             else:

#                 training_pairs.append({
#                     "anchor": anchor_text,
#                     "positive": pairs_content[:100] + "...",
#                     "negative": "Failed to parse",
#                     "anchor_id": index
#                 })
#         except json.JSONDecodeError:
#             training_pairs.append({
#                 "anchor": anchor_text,
#                 "positive": pairs_content,
#                 "negative": "JSON parse error",
#                 "anchor_id": index
#             })

#     except Exception as e:
#         print(f"Error processing row {index}: {e}")
#         training_pairs.append({
#             "anchor": anchor_text,
#             "positive": "Generation failed",
#             "negative": "Generation failed",
#             "anchor_id": index
#         })


# with open('sentence_transformer_pairs.json', 'w', encoding='utf-8') as f:
#     json.dump(training_pairs, f, indent=2, ensure_ascii=False)

# print(f"Generated {len(training_pairs)} anchor-positive-negative triplets")
# print("Sample triplet:")
# if training_pairs:
#     sample = training_pairs[0]
#     print(f"Anchor: {sample['anchor']}")
#     print(f"Positive: {sample['positive']}")
#     print(f"Negative: {sample['negative']}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Processing row 0: Oscar Kilo Papa Mike Bravo descend flight level on...
Processing row 1: Oscar Kilo Kilo Echo Alfa Praha Radar identified c...
Processing row 2: Ryanair Seven Three Alpha Hotel turn left heading ...


KeyboardInterrupt: 

## How Sentence Transformers models work


In [ ]:
from sentence_transformers import SentenceTransformer, models

## Step 1: use an existing language model
word_embedding_model = models.Transformer('distilroberta-base')

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## How to prepare your dataset for training a Sentence Transformers model


In [ ]:
%%capture
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset_id = "embedding-data/QQP_triplets"
# dataset_id = "embedding-data/sentence-compression"

dataset = load_dataset(dataset_id)

Using custom data configuration embedding-data--QQP_triplets-ff67885711b8d7f7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/embedding-data___json/embedding-data--QQP_triplets-ff67885711b8d7f7/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(f"- The {dataset_id} dataset has {dataset['train'].num_rows} examples.")
print(f"- Each example is a {type(dataset['train'][0])} with a {type(dataset['train'][0]['set'])} as value.")
print(f"- Examples look like this: {dataset['train'][0]}")

- The embedding-data/QQP_triplets dataset has 101762 examples.
- Each example is a <class 'dict'> with a <class 'dict'> as value.
- Examples look like this: {'set': {'query': 'Why in India do we not have one on one political debate as in USA?', 'pos': ['Why cant we have a public debate between politicians in India like the one in US?'], 'neg': ['Can people on Quora stop India Pakistan debate? We are sick and tired seeing this everyday in bulk?', 'Why do politicians, instead of having a decent debate on issues going in and around the world, end up fighting always?', 'Can educated politicians make a difference in India?', 'What are some unusual aspects about politics and government in India?', 'What is debate?', 'Why does civic public communication and discourse seem so hollow in modern India?', 'What is a Parliamentary debate?', "Why do we always have two candidates at the U.S. presidential debate. yet the ballot has about 7 candidates? Isn't that a misrepresentation of democracy?", 'Wh

Convert the examples into `InputExample`s. It might around 10 seconds in Google Colab.

In [ ]:
from sentence_transformers import InputExample

train_examples = []
train_data = dataset['train']['set']
# For agility we only 1/2 of our available data
n_examples = dataset['train'].num_rows // 2

for i in range(n_examples):
  example = train_data[i]
  train_examples.append(InputExample(texts=[example['query'], example['pos'][0], example['neg'][0]]))

In [ ]:
print(f"We have a {type(train_examples)} of length {len(train_examples)} containing {type(train_examples[0])}'s.")

We have a <class 'list'> of length 50881 containing <class 'sentence_transformers.readers.InputExample.InputExample'>'s.


We wrap our training dataset into a Pytorch `Dataloader` to shuffle examples and get batch sizes.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

## Loss functions for training a Sentence Transformers model


In [ ]:
from sentence_transformers import losses

train_loss = losses.TripletLoss(model=model)

## How to train a Sentence Transformer model


In [ ]:
num_epochs = 10

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

Training takes around 45 minutes with a Google Colab Pro account. Decrease the number of epochs and examples if you are using a free account or no GPU.

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3181 [00:00<?, ?it/s]

## How to share a Sentence Transformers to the Hugging Face Hub

In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
model.save_to_hub(
    "distilroberta-base-sentence-transformer",
    organization="embedding-data",
    train_datasets=["embedding-data/QQP_triplets"],
    exist_ok=True,
    )

## Extra: How to fine-tune a Sentence Transformer model


Now we will fine-tune our Sentence Transformer model.

In [ ]:
modelB = SentenceTransformer('embedding-data/distilroberta-base-sentence-transformer')

In [ ]:
dataset_id = "embedding-data/sentence-compression"
datasetB = load_dataset(dataset_id)

In [ ]:
print(f"Examples look like this: {datasetB['train']['set'][0]}")

In [ ]:
train_examplesB = []
train_dataB = dataset['train']['set']
n_examples = dataset['train'].num_rows

for i in range(n_examples):
  example = train_dataB[i]
  train_examplesB.append(InputExample(texts=[example[0], example[1]]))

In [ ]:
train_dataloaderB = DataLoader(train_examplesB, shuffle=True, batch_size=64)
train_lossB = losses.MultipleNegativesRankingLoss(model=modelB)
num_epochsB = 10
warmup_stepsB = int(len(train_dataloaderB) * num_epochsB * 0.1) #10% of train data

In [ ]:
modelB.fit(train_objectives=[(train_dataloaderB, train_lossB)],
          epochs=num_epochsB,
          warmup_steps=warmup_stepsB)

In [ ]:
modelB.save_to_hub(
    "distilroberta-base-sentence-transformer",
    organization="embedding-data",
    train_datasets=["embedding-data/sentence-compression"],
    exist_ok=True,
    )